In [ ]:
"""
    Afshin Karimi
    Std No: 99210431
    HW3 - Question 6
"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from tqdm import notebook
from sklearn.preprocessing import MinMaxScaler
import os

In [ ]:
CUDA = torch.cuda.is_available()

device = torch.device("cuda") if CUDA else torch.device("cpu")

datasets = "/content/Datasets/"

label_scalers, train_x, test_x, test_y = {}, [], {}, {}


for file in notebook.tqdm(os.listdir(datasets)): 
    # print(file)
    df = pd.read_csv(datasets + file, parse_dates=[0])
    df['hour'] = df.apply(lambda x: x['Datetime'].hour,axis=1)
    df['dayofweek'] = df.apply(lambda x: x['Datetime'].dayofweek,axis=1)
    df['month'] = df.apply(lambda x: x['Datetime'].month,axis=1)
    df['dayofyear'] = df.apply(lambda x: x['Datetime'].dayofyear,axis=1)
    df = df.sort_values("Datetime").drop("Datetime",axis=1)
    
    # Scaling 
    sc = MinMaxScaler()
    label_sc = MinMaxScaler()
    data = sc.fit_transform(df.values)
    label_sc.fit(df.iloc[:,0].values.reshape(-1,1))
    label_scalers[file] = label_sc
    
    # lookback period
    lookback = 90
    inputs = np.zeros((len(data)-lookback,lookback,df.shape[1]))
    labels = np.zeros(len(data)-lookback)
    
    for i in range(lookback, len(data)):
        inputs[i-lookback] = data[i-lookback:i]
        labels[i-lookback] = data[i,0]
    inputs = inputs.reshape(-1,lookback,df.shape[1])
    labels = labels.reshape(-1,1)
    
    # Split data into train/test portions 90:10
    test_portion = int(0.1*len(inputs))
    if len(train_x) == 0:
        train_x = inputs[:-test_portion]
        train_y = labels[:-test_portion]
    else:
        train_x = np.concatenate((train_x,inputs[:-test_portion]))
        train_y = np.concatenate((train_y,labels[:-test_portion]))
    test_x[file] = (inputs[-test_portion:])
    test_y[file] = (labels[-test_portion:])

In [ ]:
batch_size = 1024

train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)

In [ ]:
class GRU_Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(GRU_Net, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        return hidden

In [ ]:
class LSTM_Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(LSTM_Net, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.lstm(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [ ]:
def train(train_loader, learn_rate, hidden_dim, EPOCHS, model_type):
    
    input_dim = next(iter(train_loader))[0].shape[2]
    output_dim = 1
    n_layers = 2
    if model_type == "GRU":
        model = GRU_Net(input_dim, hidden_dim, output_dim, n_layers)
    else:
        model = LSTM_Net(input_dim, hidden_dim, output_dim, n_layers)
    model.to(device)
    
    # Defining loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)
    
    model.train()
    print(f"Training of {model_type} model")
    epoch_times = []
    # Start training loop
    for epoch in range(1,EPOCHS+1):
        h = model.init_hidden(batch_size)
        avg_loss = 0.
        counter = 0
        for x, label in train_loader:
            counter += 1
            if model_type == "GRU":
                h = h.data
            else:
                h = tuple([e.data for e in h])
            model.zero_grad()
            
            out, h = model(x.to(device).float(), h)
            loss = criterion(out, label.to(device).float())
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            if counter%200 == 0:
                print(f"Epoch {epoch}---Step: {counter}/{len(train_loader)}--- Average Loss for Epoch: {avg_loss/counter}")
        print(f"Epoch {epoch}/{EPOCHS}, Total Loss: {avg_loss/len(train_loader)}")
    return model

def evaluate(model, test_x, test_y, label_scalers):
    model.eval()
    outputs = []
    targets = []
    for i in test_x.keys():
        inp = torch.from_numpy(np.array(test_x[i]))
        labs = torch.from_numpy(np.array(test_y[i]))
        h = model.init_hidden(inp.shape[0])
        out, h = model(inp.to(device).float(), h)
        outputs.append(label_scalers[i].inverse_transform(out.cpu().detach().numpy()).reshape(-1))
        targets.append(label_scalers[i].inverse_transform(labs.numpy()).reshape(-1))
    sMAPE = 0
    for i in range(len(outputs)):
        sMAPE += np.mean(abs(outputs[i]-targets[i])/(targets[i]+outputs[i])/2)/len(outputs)
    print(f"sMAPE: {sMAPE*100}%")
    return outputs, targets, sMAPE

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
EPOCH = 5
HIDDEN_DIM = 256
lr = 0.001
gru_model = train(train_loader, lr, HIDDEN_DIM, EPOCH, model_type="GRU")

Training of GRU model
Epoch 1---Step: 200/957--- Average Loss for Epoch: 0.005976225932827219
Epoch 1---Step: 400/957--- Average Loss for Epoch: 0.0033440310122387017
Epoch 1---Step: 600/957--- Average Loss for Epoch: 0.0023793668135476762
Epoch 1---Step: 800/957--- Average Loss for Epoch: 0.001867244483091781
Epoch 1/5, Total Loss: 0.0016037222776085608
Epoch 2---Step: 200/957--- Average Loss for Epoch: 0.00023916856189316605
Epoch 2---Step: 400/957--- Average Loss for Epoch: 0.0002287233066454064
Epoch 2---Step: 600/957--- Average Loss for Epoch: 0.00022113338425697293
Epoch 2---Step: 800/957--- Average Loss for Epoch: 0.000214178673850256
Epoch 2/5, Total Loss: 0.00020754416950974933
Epoch 3---Step: 200/957--- Average Loss for Epoch: 0.0001698205182037782
Epoch 3---Step: 400/957--- Average Loss for Epoch: 0.00016011264957342065
Epoch 3---Step: 600/957--- Average Loss for Epoch: 0.00015565061798649064
Epoch 3---Step: 800/957--- Average Loss for Epoch: 0.00015345253521445557
Epoch 3/5

In [ ]:
lstm_model = train(train_loader, lr, HIDDEN_DIM, EPOCH, model_type="LSTM")

Training of LSTM model
Epoch 1---Step: 200/957--- Average Loss for Epoch: 0.010106340588536114
Epoch 1---Step: 400/957--- Average Loss for Epoch: 0.005632541824888903
Epoch 1---Step: 600/957--- Average Loss for Epoch: 0.003969829769678957
Epoch 1---Step: 800/957--- Average Loss for Epoch: 0.0030907433351967484
Epoch 1/5, Total Loss: 0.0026376759473761384
Epoch 2---Step: 200/957--- Average Loss for Epoch: 0.00028307295469858217
Epoch 2---Step: 400/957--- Average Loss for Epoch: 0.00025835778174950976
Epoch 2---Step: 600/957--- Average Loss for Epoch: 0.00024469337511012177
Epoch 2---Step: 800/957--- Average Loss for Epoch: 0.00023240377367983456
Epoch 2/5, Total Loss: 0.0002243095532682741
Epoch 3---Step: 200/957--- Average Loss for Epoch: 0.00017906698791193777
Epoch 3---Step: 400/957--- Average Loss for Epoch: 0.00017485753938672132
Epoch 3---Step: 600/957--- Average Loss for Epoch: 0.00017055375114675067
Epoch 3---Step: 800/957--- Average Loss for Epoch: 0.00016609126722869405
Epoch 

In [ ]:
gru_outputs, targets, gru_sMAPE = evaluate(gru_model, test_x, test_y, label_scalers)

sMAPE: 0.27756364212935497%


In [ ]:
lstm_outputs, targets, lstm_sMAPE = evaluate(lstm_model, test_x, test_y, label_scalers)

sMAPE: 0.24942247827065064%
